In this example we will enhance an AI Agent with self-reflection capabilities, allowing it to critique its own responses and refine them iteratively. This feature enables the agent to evaluate its own output and improve the response quality before delivering its final answer.

Objective

The task is to modify the agent so that it can:

    -Store conversation history - implement a memory mechanism to track interactions
    -Generate an initial response - Process user input and return a response using the language model
    -Critique its own response when enabled - if self-reflection is activated, the agent should generate feedback on its own answer
    -Refine its response iteratively - Based on the self critique, the agent should adjust its reply, improving clarity, accuracy, and relevance

Steps

    1) Imlement a memory layer to retain conversation history
    2) Introduce a self-reflection mechanism that allows the agent to analyze its response and refine it
    3) Limit the number of self-reflection iterations to prevent excesssive loops (min=1, max=3)
    4) Ensure flexibility by allowing users to toggle self-reflection on or off

Considerations

    -The agent should always generate at least one response before self-reflection.
    -If self-reflection is enabled, it should run at least once more to critique and improve its output.
    -The number of iterations should be controlled and not exceed three refinements
    -Implement logging functionality (verbose mode) to track the refinement process

Import the necessary libraries

In [ ]:
import json
from typing import List, Dict, Literal
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
client = OpenAI()

In [ ]:
response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Answer all questions."},
                    {"role": "user", "content": "What have i asked?"},
                ],
                temperature=0.1,
        )
response.choices[0].message.content

Add the memory layer: to add reflection, our agent has to be able to keep track of interactions

In [ ]:
memory = [
    {"role": "system", "content": "Answer all user questions"},
    {"role": "user", "content": "whats an LLM?"},
]

In [ ]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.1,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

In [ ]:
memory.append(
    {"role": "user", "content": "what have i asked?"}
)

memory

In [ ]:
new_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=memory,
    temperature=0.1,
)

memory.append(
    {"role": "assistant", "content": new_response.choices[0].message.content}
)

memory

We need to create a proper memory class to deal with more complicated cases

We will create the memory class with the following methods:

    -add_message
    -get_messages
    -last_message

In [ ]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []

    def add_message(self, role: Literal['user', 'system', 'assistant'], content: str):
        self._messages.append({
            "role": role,
            "content": content
        })
    
    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages
    
    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

INVOKE

refactor invoke() method. this method should now include:

    -self-reflection parameter (default: False)
    -max_iter parameter (default: 1)

If self_reflection is set to true, it should use a loop to generate an initial response. Then critiquing and refining the response in subsequent iterations up to the number of iterations defined in max_iter.

Use the self.memory to store each step

Rules for Self-Reflection:

    -Dont allow values less than 1
    -Dont allow values greater than 3
    -Max iter is controlled by self-reflection flag
    -If set to true, it needs to call the LLM at least once more for the criticism

In [ ]:
self_critique_prompt = """
Look at your last response and take some time to think about it. Are there any mistakes? Is there a way to improve the response or better ways to clarify
the message you are trying to get across?
Provide a revised response, if necessary, in a JSON output structure:
{
    "original_response": "",
    "revisions_needed": "",
    "updated_response": ""
}
"""

In [ ]:
class Agent:
    """A self-reflection AI agent"""

    def __init__(
        self,
        name:str = "Agent",
        role:str = "Personal Assistant",
        instructions:str = "Help users with any questions",
        model:str = "gpt-4o-mini",
        temperature:float = 0.1,
    ):
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature

        self.client = OpenAI()

        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"Youre an AI agent, your role is {self.role}, "
                f"and you need to {self.instructions}",
        )

        self.critique_prompt = self_critique_prompt

    def invoke(self,
                user_message: str,
                self_reflection: bool = False,
                max_iter: int = 1,
                verbose: bool = False) -> str:
    
        self.memory.add_message(
            role="user",
            content=user_message
        )
        if verbose:
            self._log_last_message()

        max_iter = max_iter if max_iter >= 1 else 1
        max_iter = max_iter if max_iter <= 3 else 3
        max_iter = max_iter if self_reflection else 0.5
        loops = 2 * max_iter

        for i in range(loops):
            ai_message = self._get_completion(
                messages = self.memory.get_messages()
            )

            self.memory.add_message(
                role = "assistant",
                content = ai_message.content,
            )
            if verbose:
                self._log_last_message()

            if i < loops - 1:
                self.memory.add_message(
                    role = "user",
                    content = self.critique_prompt
                )
                if verbose:
                    self._log_last_message()
                
                ai_message = self._get_completion(
                    messages = self.memory.get_messages()
                )

    def _get_completion(self, messages:List[Dict])-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages
        )

        return response.choices[0].message
    
    def _log_last_message(self):
        print(f"### {self.memory.last_message()['role']} message ###\n".upper())
        print(f"{self.memory.last_message()['content']} \n")
        print("\n_____________________________________________________\n")

Now we can test out some agents

In [ ]:
agent = Agent()
agent.invoke(
    user_message="Who is the greatest basketball player of all time? Lebron James or Michael Jordan?",
    self_reflection=True,
    verbose=True,
)

In [ ]:
agent.memory.get_messages()